# News headline text generation model

Here, I create a model that generates news headlines using the Kaggle dataset of New York Times comments and headlines. 

## 1) import the necessary libraries.

In [1]:
# necessary imports

# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

# set seeds for reproducability
from tensorflow import set_random_seed
from numpy.random import seed
set_random_seed(2)
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

Using TensorFlow backend.


## 2) Load the [dataset](https://www.kaggle.com/aashita/nyt-comments) of New York Times comments and headlines.

In [4]:
# method from Kaggle kernel

curr_dir = '/Users/laurenfinkelstein/take_home_tests/SynapseFI/data/'
all_headlines = []
for filename in os.listdir(curr_dir):
    if 'Articles' in filename:
        article_df = pd.read_csv(curr_dir + filename)
        all_headlines.extend(list(article_df.headline.values))
        break

all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

831

In [ ]:
# William method? using glob?

## 3) Prepare the data

### 1. Clean the data

- remove punctuation
- lowercase all words

In [6]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower() # removes punctuation and lowercases words
    txt = txt.encode("utf8").decode("ascii",'ignore') # converts from utf8 to ascii -- REMOVE THIS!!
    return txt 

corpus = [clean_text(x) for x in all_headlines]
corpus[:10]

['finding an expansive view  of a forgotten people in niger',
 'and now  the dreaded trump curse',
 'venezuelas descent into dictatorship',
 'stain permeates basketball blue blood',
 'taking things for granted',
 'the caged beast awakens',
 'an everunfolding story',
 'oreilly thrives as settlements add up',
 'mouse infestation',
 'divide in gop now threatens trump tax plan']

## 2. Generate sequence of n-gram tokens

Language modelling takes in sequential data (i.e., words/tokens), and uses this sequential data to predict the next word/token. This requires tokenization, which is the process of extracting tokens (i.e., terms/words) from a corpus. To do this, I use the Python library Keras' built-in model for tokenization, which is able to obtain the tokens and their respsective index in the corpus. This allows for every text document in the dataset to be converted into a sequence of tokens.

In [10]:
# create the tokenizer
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus) # fit the tokenizer on the documents
    total_words = len(tokenizer.word_index) + 1 # word index gives a dictionary of words and their uniquely assigned integers
    
    ## convert data to sequence of tokens 
    input_sequences = [] # sequence of n-grams from all documents in the corpus
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0] # list of tokens corresponding to each word in the document (i.e., news headline)
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence) # sequence of n-grams (from 2 word n-grams, through n-grams = len(line))
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:10]

[[169, 17],
 [169, 17, 665],
 [169, 17, 665, 367],
 [169, 17, 665, 367, 4],
 [169, 17, 665, 367, 4, 2],
 [169, 17, 665, 367, 4, 2, 666],
 [169, 17, 665, 367, 4, 2, 666, 170],
 [169, 17, 665, 367, 4, 2, 666, 170, 5],
 [169, 17, 665, 367, 4, 2, 666, 170, 5, 667],
 [6, 80]]

Each list in the above output represents the ngram phrases generated from the input data (i.e., the documents/headlines). Each integer in these n-grams is the index of the given word in the complete vocabulary of words present in the corpus of text. 

For example, the first headline in the corpus is "Finding an expansive view of a forgotten people in niger". For this headline, we see the following output, i.e. **sequences of tokens**:

    [169, 17],
    [169, 17, 665],
    [169, 17, 665, 367],
    [169, 17, 665, 367, 4],
    [169, 17, 665, 367, 4, 2],
    [169, 17, 665, 367, 4, 2, 666],
    [169, 17, 665, 367, 4, 2, 666, 170],
    [169, 17, 665, 367, 4, 2, 666, 170, 5],
    [169, 17, 665, 367, 4, 2, 666, 170, 5, 667]

which, respectively, equate to the following **n-grams**: 

    Finding an,
    Finding an expansive,
    Finding an expansive view,
    Finding an expansive view of,
    Finding an expansive view of a,
    Finding an expansive view of a forgotten,
    Finding an expansive view of a forgotten people,
    Finding an expansive view of a forgotten people in,
    Finding an expansive view of a forgotten people in niger

## 3. Pad the sequences and obtain variables (predictor and target)

Up to this point, we've generated a data-set containing a sequence of tokens. It's important to recognize that different sequences may have different lengths. Before training the model, we must ensure all sequence lengths are equal. For this, we can pad the sequences using the Keras pad_sequence function. 

Additionally, inorder to input data into the model, we must create predictors and a label. We will use a n-grams sequence as predictors, and the next word of the n=gram as the label. 

For example, for the headline "Finding an expansive view of a forgotten people in niger", the **predictors** will be:

    Finding an
    Finding an expansive
    Finding an expansive view

and the corresponding **labels** will be

    expansive
    view
    of

... and so on. 

In [14]:
def generate_padded_sequences(input_sequences):
    ## pad the sequences
    max_sequence_len = max([len(x) for x in input_sequences]) # find length of the longest input sequence (i.e., headline)
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')) # pads the input sequences and converts to a numpy array
    ## ^ WHY IS IT WRAPPED AS NP.ARRAY? WHEN I TEST PRINT IT WITH/WITHOUT NP.ARRAY(), OUTPUT IS THE SAME
    
    ## create predictors and label
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1] # label is the last word of each n-gram, and predictors are all the preceeding words
    label = ku.to_categorical(label, num_classes=total_words) # one-hot encodes the labels; this is a matrix of 4,806 rows (the number of documents/headlines in the corpus) and 2,422 columns (the number of words/tokens in the corpus (i.e., length of input_sequences))
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

Now we can obtain the input vector *X* and the label vector _Y_, which will be used for to training the network. 

Here we will use a LSTM model with a three layer architecture, including:
    
***Input Layer*** : Takes the sequence of words as input
<br /> ***LSTM Layer*** : Computes the output using LSTM units. There are currently 100 units, but this can be fine-tuned later.
<br /> ***Dropout Layer*** : A regularization layer which randomly turns off the activations of some neurons in the LSTM layer in order to prevent overfitting. (Note: this is an optional layer)
<br /> ***Output Layer*** : Computes the probability of the best possible next word as output
    
We will run this model for  100 epochs, but this can be further experimented with and fine-tuned. 

In [15]:
## create the model

def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1 # - 1 because the last word in the sequence is used as the label (?)
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 18, 10)            24220     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2422)              244622    
Total params: 313,242
Trainable params: 313,242
Non-trainable params: 0
_________________________________________________________________


Now that we've created the model architecture, we can train it using our data (predictors and labels). 

In [16]:
## train the model

model.fit(predictors, label, epochs=100, verbose=5)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100

## 5) Generate text

Now that the model has been trained, we can use it to predict the next word based on input words (i.e., seed text). 

To feed this seed text into the model, we will need to do some pre-processing, which includes (1) tokenizing the seed text and (2) padding the sequences. We can then pass the sequences into the trained model to get the predicted word. The multiple predicted words can then be appended together to obtain a predicted sequence. 

In [17]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0] # list of tokens corresponding to each word in the document (i.e., news headline)
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre') # add padding
        predicted = model.predict_classes(token_list, verbose=0) # make predictions using trained model
        
        output_word = ""
        for word,index in tokenizer.word_index.items(): # word index gives a dictionary of words and their uniquely assigned integers
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

## 6) Some results

In [18]:
print (generate_text("united states", 5, model, max_sequence_len))
print (generate_text("preident trump", 4, model, max_sequence_len))
print (generate_text("donald trump", 4, model, max_sequence_len))
print (generate_text("india and china", 4, model, max_sequence_len))
print (generate_text("new york", 4, model, max_sequence_len))
print (generate_text("science and technology", 5, model, max_sequence_len))

United States Governor Bands Is Youths Is
Preident Trump Officials Brace For Anger
Donald Trump Country Shock On Trump
India And China Deep Flavor That Nafta
New York Today Sharing A Block
Science And Technology Shut From A Clash Jet


# Improvement Ideas
As we can see, the model has produced the output which looks fairly fine. The results can be improved further with following points:

- Adding more data
- Fine Tuning the network architecture
- Fine Tuning the network parameters